In [1]:
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

from scipy.stats import chi2_contingency

from pipelines import preprocessor, preprocessor2
from preprocess import extra_features

In [2]:
train=pd.read_csv('Datasets/Train.csv')
train=extra_features(train)
X=train.drop('cost_category', axis=1)
y= train['cost_category']

<IPython.core.display.Javascript object>

In [3]:
col_drop=[]

for col in X.columns:

    ct=pd.crosstab(y, X[col])
    chi2, p, _, _ =chi2_contingency(ct)

    if p >= 0.05:
        col_drop.append(col)
    elif chi2 <= 3000:
        col_drop.append(col)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
X.drop(columns=col_drop, inplace=True)

In [5]:
X=preprocessor2(X)

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
len(le.classes_)

6

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
model=CatBoostClassifier(n_estimators=600,
                        max_depth=16,
                        learning_rate=0.1,
                        use_best_model=True,
                        eval_metric='Accuracy',
                        )

model.fit(X_train, y_train, eval_set=(X_test,y_test))

0:	learn: 0.5206701	test: 0.5108050	best: 0.5108050 (0)	total: 41.6ms	remaining: 24.9s
1:	learn: 0.5294515	test: 0.5140465	best: 0.5140465 (1)	total: 115ms	remaining: 34.5s
2:	learn: 0.5455958	test: 0.5216099	best: 0.5216099 (2)	total: 115ms	remaining: 34.5s
3:	learn: 0.5538368	test: 0.5264722	best: 0.5264722 (3)	total: 115ms	remaining: 34.5s
4:	learn: 0.5606593	test: 0.5256618	best: 0.5264722 (3)	total: 115ms	remaining: 34.5s
5:	learn: 0.5686976	test: 0.5305240	best: 0.5305240 (5)	total: 115ms	remaining: 34.5s
6:	learn: 0.5756552	test: 0.5348460	best: 0.5348460 (6)	total: 115ms	remaining: 34.5s
7:	learn: 0.5794380	test: 0.5351162	best: 0.5351162 (7)	total: 115ms	remaining: 34.5s


In [20]:
y_pred=model.predict(X_test)

In [21]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.32      0.48      0.38       498
           1       0.73      0.58      0.65      1206
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.59      0.55      0.57       522
           5       0.71      0.54      0.61      1476

    accuracy                           0.55      3702
   macro avg       0.39      0.36      0.37      3702
weighted avg       0.65      0.55      0.59      3702



C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
